In [1]:
import nltk
import gensim
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np

file_docs = []

with open ('pdf2.txt') as f:
    for sentence in gensim.summarization.textcleaner.get_sentences(f.read()):
        file_docs.append(sentence)

print("Number of documents:",len(file_docs))

Number of documents: 785


In [17]:
file2_docs = []

with open ('pdf1.txt') as f:
    text=f.read()
    text = text.replace("\n", "")
    file2_docs=gensim.summarization.textcleaner.split_sentences(text)
    
print("Number of documents:",len(file2_docs))

Number of documents: 269


In [19]:
with open('nltk_eng_stopwords.txt', 'r') as f:
    stopwords = [line.strip() for line in f]

In [29]:
prepros1 = [[w.lower() for w in word_tokenize(text)] for text in file_docs if text not in stopwords]
prepros2 = [[w.lower() for w in word_tokenize(text)] for text in file2_docs if text not in stopwords]

In [31]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

In [32]:
# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

In [33]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(prepros1+prepros2)
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

In [35]:
query_tf = tfidf[[dictionary.doc2bow(document) for document in prepros2]]

sim = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in prepros1]],
            similarity_matrix)

doc_similarity_scores = sim[query_tf]

In [36]:
doc_similarity_scores

array([[0.32331762, 0.14282295, 0.13675347, ..., 0.21774076, 0.46133253,
        0.24338405],
       [0.3573962 , 0.13500562, 0.1060555 , ..., 0.21132448, 0.44006768,
        0.10015951],
       [0.24968882, 0.06982826, 0.07999936, ..., 0.2791498 , 0.33459347,
        0.03054112],
       ...,
       [0.1690618 , 0.1367577 , 0.15195134, ..., 0.1640106 , 0.4272504 ,
        0.26984048],
       [0.2702787 , 0.15113078, 0.11654562, ..., 0.2910969 , 0.4542206 ,
        0.10064195],
       [0.50038916, 0.15642236, 0.10713472, ..., 0.18716566, 0.43223143,
        0.28897578]], dtype=float32)

In [37]:
max_list=[]
i=1
N=doc_similarity_scores.copy()

while i<20:
    max_value, max_index = max(((x, (i, j)) for i, row in enumerate(N) for j, x in enumerate(row)))
    file2=file2_docs[max_index[0]]
    file1=file_docs[max_index[1]]
    tup=(max_value, max_index,file2,file1)
    max_list.append(tup)
    N = [[0 if j==max_value else j for j in i] for i in N]
    i=i+1

In [38]:
import pandas as pd
df=pd.DataFrame(max_list,columns=['similarity','location','file2','file1'])

In [41]:
pd.set_option('display.max_colwidth', -1)
df[['similarity','file2','file1']]

,similarity,file2,file1
0,0.793777,"In this header compression, if the header can be decompressed with its sequence number, the reference information is not updated, and only the sequence number is included in the header.",Each header in a string can be compressed by representing it with a ROHC header which essentially only carries an encoded sequence number.
1,0.763338,"Also, even if one decompression error is detected, it is impossible to tell the cause of the decompression error, that is, either noise or erroneous reference information.","If links were made (almost) error free, the system might not be able to support a sufficiently large number of users per cell and might thus be economically infeasible."
2,0.741349,H'1 = H1* (1).,"3) k is calculated as k = ceiling(log2(2 * J + 1), where J = Max_Jitter_BC + Max_Jitter_CD + 2."
3,0.739527,"In this header compression, if the header can be decompressed with its sequence number, the reference information is not updated, and only the sequence number is included in the header.",String A sequence of headers in which the values of all fields being compressed change according to a pattern which is fixed with respect to a sequence number.
4,0.737392,"[0018] In some cases, however, the header is not regarded as having an error even if it has not been correctly decompressed, and therefore the packet is not discarded.","While the latter is true, the kind of loss induced by CRTP is difficult to deal with."
5,0.736196,"If such increase in size happens quite often, the transmission path is substantially decreased in effective speed.",The compression efficiency is determined by how much the header sizes are reduced by the compression scheme.
6,0.733487,11.,Standards Track [Page 28]RFC 3095 Robust Header Compression July 2001 cover 20 ms.
7,0.720523,"Also, even if one decompression error is detected, it is impossible to tell the cause of the decompression error, that is, either noise or erroneous reference information.","It seems reasonable to believe that in the years to come, IP will become a commonly used way to carry telephony."
8,0.716721,How to compress the header will be described later.,"While the latter is true, the kind of loss induced by CRTP is difficult to deal with."
9,0.715943,"If the header cannot be decompressed with its sequence number, update information of the reference information and the sequence number are included in the header.",String A sequence of headers in which the values of all fields being compressed change according to a pattern which is fixed with respect to a sequence number.
